In [1]:
import pandas as pd
from pyserini.search.lucene import LuceneSearcher
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import string

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [4]:
stop_words = set(stopwords.words('indonesian'))

In [5]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [6]:
def normalize_text(text, dictionary):
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [7]:
def preprocess_query(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ''.join([char for char in text if char not in string.punctuation])
    text = normalize_text(text, istilah_kesehatan)
    tokens = text.split()
    stemmed_tokens = stem_tokens(tokens)
    
    return ' '.join(stemmed_tokens)

In [8]:
queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'kesehatan mental',
  'efek samping vaksin covid-19',
  'manfaat buah'
]

In [9]:
queries = [preprocess_query(query) for query in queries]

In [10]:
print(queries)

['gejala diabetes', 'makan sehat jantung', 'sehat mental', 'efek samping vaksin covid19', 'manfaat buah']


# Membaca Dataframe

In [25]:
df = pd.read_csv('../artikel/raw_health_articles.csv')
df.drop(columns=['description', 'content'], inplace=True)
df.head()

,article_index,title,tag,link,date,source
0,1,5 Cara Merawat Ginjal agar Sehat & Cegah Penyakit Ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,Biofarma
1,2,7 Tips Untuk Menjaga Kesehatan Mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,Biofarma
2,3,Jangan Sepelekan! 10 Penyebab Campak yang Sering Terabaikan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,Biofarma
3,4,"Campak - Penyebab, Gejala, dan Pencegahannya",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,Biofarma
4,5,Cara Mudah Menjaga Kesehatan Gigi dan Mulut agar Terhindar dari Penyakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,Biofarma


In [12]:
# Fungsi untuk print hasil
def display_results(query, hits):
    print(f'\n{query}')

    if not hits:
        print("Tidak ada dokumen yang sesuai.")
        return
    
    results_with_scores = []
    
    for i in range(len(hits)):
        matching_row = df[df['article_index'] == int(hits[i].docid)].copy()
        
        if not matching_row.empty:
            # Add score to the dataframe row
            matching_row = matching_row.copy()
            matching_row['score'] = hits[i].score
            matching_row['rank'] = i + 1
            results_with_scores.append(matching_row)
            
            # print(f'{i+1:2} ID: {hits[i].docid:4} Score: {hits[i].score:.5f}')
            # print(f'   Title: {matching_row.iloc[0]["title"]}')
            # print()
    
    if results_with_scores:
        result_df = pd.concat(results_with_scores, ignore_index=True)
        return result_df
    else:
        print("No matching documents found in DataFrame.")
        return pd.DataFrame()

## Bm25 Retrieval (Title)

In [13]:
searcher_title = LuceneSearcher('../indexing/indexes/title_jsonl')
searcher_content_title = LuceneSearcher('../indexing/indexes/title_content_jsonl')
searcher_desc_content = LuceneSearcher('../indexing/indexes/title_desc_content_jsonl')

In [26]:
# Test the function with a query
query = queries[0]  
hits = searcher_title.search(query)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


gejala diabetes

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,Gejala Diabetes Melitus yang Harus Kamu Ketahui!,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/gejala-diabetes-melitus-yang-harus-kamu-ketahui,11 Desember 2024,Biofarma,3.715400,1
1,410,"Gejala Diabetes Tipe 1 pada Anak, Banyak Minum hingga Berat Badan Turun",HEALTH,https://health.kompas.com/read/25I12181500168/gejala-diabetes-tipe-1-pada-anak-banyak-minum-hingga-berat-badan-turun,12 September 2025,kompas,3.203300,2
2,49,5 Faktor Risiko Diabetes yang Perlu Anda Ketahui,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-faktor-risiko-diabetes-yang-perlu-anda-ketahui,25 Maret 2025,Biofarma,2.141600,3
3,51,Bahaya Diabetes Mellitus dan Faktor Risiko yang Meningkatkan Kemungkinannya,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/bahaya-diabetes-mellitus-dan-faktor-risiko-yang-meningkatkan-kemungkinannya,25 Maret 2025,Biofarma,2.077200,4
4,201,"Hari Diabetes Sedunia, Diabetasol Gelar Edukasi Kesehatan",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/hari-diabetes-sedunia-diabetasol,Oktober November 2022,Klikdokter,2.016600,5
5,367,"7 Camilan Enak untuk Pengidap Diabetes, Termasuk Alpukat dan Almond",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8113648/7-camilan-enak-untuk-pengidap-diabetes-termasuk-alpukat-dan-almond,15 September 2025,detik,2.016599,6
6,414,"Diabetes Tipe 1 pada Anak, Pentingnya Diagnosis Dini untuk Cegah Kematian",HEALTH,https://health.kompas.com/read/25I11191500768/diabetes-tipe-1-pada-anak-pentingnya-diagnosis-dini-untuk-cegah-kematian,11 September 2025,kompas,2.016598,7
7,353,Efek Samping Metformin pada Pengguna Diabetes Tipe 2 dan Tips Pencegahannya,KESEHATAN,https://www.alodokter.com/efek-samping-metformin-pada-pengguna-diabetes-tipe-2-dan-tips-pencegahannya,11 September 2025,alodokter,1.905400,8
8,103,"Stres - Gejala, Penyebab, dan Pencegahan",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/stres-gejala-penyebab-dan-pencegahan,21 Oktober 2024,Biofarma,1.505400,9
9,4,"Campak - Penyebab, Gejala, dan Pencegahannya",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,Biofarma,1.505399,10


## Bm25 Retrieval (Title + Content)

In [31]:
# Test the function with a query
query = queries[4] 
hits = searcher_content_title.search(query, k=10)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


manfaat buah

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,222,Apa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,halodoc,2.2503,1
1,299,7 Manfaat Buah Pinang Muda untuk Pria yang Perlu Diketahui,KESEHATAN,https://www.alodokter.com/7-manfaat-buah-pinang-muda-untuk-pria-yang-perlu-diketahui,16 September 2025,alodokter,2.2434,2
2,301,"Buah Pelancar ASI, Dukungan Alami untuk Menyusui Lebih Lancar",KELUARGA,https://www.alodokter.com/buah-pelancar-asi-dukungan-alami-untuk-menyusui-lebih-lancar,16 September 2025,alodokter,2.2166,3
3,102,4 Manfaat Buah Belimbing untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/4-manfaat-buah-belimbing-untuk-kesehatan,22 Oktober 2024,Biofarma,2.2073,4
4,58,Kenali 5 Manfaat Kurma yang Membantu Menjaga Kesehatan Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/kenali-5-manfaat-kurma-yang-membantu-menjaga-kesehatan-tubuh,12 Maret 2025,Biofarma,2.1995,5
5,81,5 Manfaat Buah Sirsak Yang Baik Bagi Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-manfaat-buah-sirsak-yang-baik-bagi-tubuh,2 Januari 2025,Biofarma,2.1988,6
6,53,Manfaat Buah Srikaya: Si Manis dengan Segudang Khasiat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-buah-srikaya-si-manis-dengan-segudang-khasiat,21 Maret 2025,Biofarma,2.1863,7
7,63,Buah Duku: Superfood Tropis yang Kaya Nutrisi dan Manfaat Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/buah-duku-superfood-tropis-yang-kaya-nutrisi-dan-manfaat-kesehatan,25 Februari 2025,Biofarma,2.1650,8
8,347,"Tomat Buah atau Sayur, Ini Penjelasannya",KESEHATAN,https://www.alodokter.com/tomat-buah-atau-sayur-ini-penjelasannya,10 September 2025,alodokter,2.1647,9
9,82,8 Ragam Manfaat Buah Naga Untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/8-ragam-manfaat-buah-naga-untuk-kesehatan,19 Desember 2024,Biofarma,2.1592,10


## Bm25 Retrieval (Title + Desc + Content)

In [28]:
# Test the function with a query
query = queries[0] 
hits = searcher_desc_content.search(query)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


gejala diabetes

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,Gejala Diabetes Melitus yang Harus Kamu Ketahui!,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/gejala-diabetes-melitus-yang-harus-kamu-ketahui,11 Desember 2024,Biofarma,2.2057,1
1,49,5 Faktor Risiko Diabetes yang Perlu Anda Ketahui,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-faktor-risiko-diabetes-yang-perlu-anda-ketahui,25 Maret 2025,Biofarma,2.1629,2
2,410,"Gejala Diabetes Tipe 1 pada Anak, Banyak Minum hingga Berat Badan Turun",HEALTH,https://health.kompas.com/read/25I12181500168/gejala-diabetes-tipe-1-pada-anak-banyak-minum-hingga-berat-badan-turun,12 September 2025,kompas,2.0167,3
3,353,Efek Samping Metformin pada Pengguna Diabetes Tipe 2 dan Tips Pencegahannya,KESEHATAN,https://www.alodokter.com/efek-samping-metformin-pada-pengguna-diabetes-tipe-2-dan-tips-pencegahannya,11 September 2025,alodokter,1.9762,4
4,201,"Hari Diabetes Sedunia, Diabetasol Gelar Edukasi Kesehatan",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/hari-diabetes-sedunia-diabetasol,Oktober November 2022,Klikdokter,1.8422,5
5,414,"Diabetes Tipe 1 pada Anak, Pentingnya Diagnosis Dini untuk Cegah Kematian",HEALTH,https://health.kompas.com/read/25I11191500768/diabetes-tipe-1-pada-anak-pentingnya-diagnosis-dini-untuk-cegah-kematian,11 September 2025,kompas,1.8149,6
6,51,Bahaya Diabetes Mellitus dan Faktor Risiko yang Meningkatkan Kemungkinannya,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/bahaya-diabetes-mellitus-dan-faktor-risiko-yang-meningkatkan-kemungkinannya,25 Maret 2025,Biofarma,1.8014,7
7,342,5 Efek Samping Minum Rebusan Daun Ciplukan yang Perlu Diwaspadai,HIDUP SEHAT,https://www.alodokter.com/5-efek-samping-minum-rebusan-daun-ciplukan-yang-perlu-diwaspadai,10 September 2025,alodokter,1.7947,8
8,317,6 Efek Samping Methylprednisolone yang Perlu Diketahui,KESEHATAN,https://www.alodokter.com/6-efek-samping-methylprednisolone-yang-perlu-diketahui,11 September 2025,alodokter,1.7118,9
9,64,Kapulaga: Rempah Ajaib dengan Segudang Manfaat untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/kapulaga-rempah-ajaib-dengan-segudang-manfaat-untuk-kesehatan,21 Februari 2025,Biofarma,1.6805,10
